## NCMF - AX
Example of running the "ncmf" module with the best parameters found using the ax framework

In [1]:
sample_no = 1
data_dir = f"../../datasets/NCMF/"
dataset_name = "MIMIC"

In [2]:
import ax
from ax import RangeParameter, ChoiceParameter, FixedParameter
from ax import ParameterType, SearchSpace
from ax.service.managed_loop import optimize

In [3]:
import sys
sys.path.append("..")

In [4]:
import pprint
import numpy as np
import pickle as pkl
import time
import itertools
import os
import pprint

In [5]:
from src.ncmf import ncmf

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

In [7]:
import torch

In [8]:
pp = pprint.PrettyPrinter()

#### *Optimization/training - hyperparamteres*

- **learning_rate**: float, Adam optimizer's learning rate
- **weight_decay**: float, Adam optimizers's weight decay (L2 penalty)
- **max_epochs**: int, maximum number of training epochs at which the training stops 
- **convg_thres**: float, convergence threshold 

# Hyperparameter selection using the ax framework

- Installation instruction can be found at: [https://ax.dev/](https://ax.dev/)
- The example below is based on the following API:
[https://ax.dev/tutorials/gpei_hartmann_loop.html](https://ax.dev/tutorials/gpei_hartmann_loop.html)
- And here is a high level intro to the library: 
[https://www.youtube.com/watch?v=2c8YX0E8Qhw](https://www.youtube.com/watch?v=2c8YX0E8Qhw)

In [9]:
# Create a wrapper method for DCMF to use with the ax framework
# Here we perform the hyper parameter optimization based on the training loss
# i.e. finding the optimum hyperparams that results in minimum loss

def run_ncmf(parameterization):
    #hyper-parameters that are selected using ax
    learning_rate = 1e-6
    convergence_threshold = 1e-3
    num_epochs = 10
    batch_size = 2048
    entity_matrices = ['X0', 'X1', 'X2']
    matrix_types = {
    "real": ["X0", "X1", "X2"],
    "binary": []
    }
    weight_decay = parameterization["weight_decay"]
    ncmf_model = ncmf(sample_no, data_dir, dataset_name, matrix_types, num_epochs, learning_rate, weight_decay, convergence_threshold, batch_size, batch_size, entity_matrices)
    #
    ncmf_model.fit()
    ncmf_model.evaluate()
    print("#")
    print("ncmf_model.out_dict_info: ")
    pp.pprint(ncmf_model.out_dict_info)
    print("#")
    #
    out_dict = {}
    out_dict["auc"] = (ncmf_model.out_dict_info["auc"], 0.0)
    return out_dict

In [10]:
# The ax method that performs the hyperparams optimization
# Here we optimize only 2 hyperparameters: learning_rate and weight_decay. 
# You can add more hyperparams as in the commented section
# The optimization is performed with 2 DCMF executions. 
# You can change this by setting "total_trials" as desired
# Tip: Use atleast total_trials=50 for finding near optimum of the two hyperparameters
num_chunks = 2
is_gpu = False
gpu_ids = "1"
from torch.utils.tensorboard import SummaryWriter
best_parameters, values, experiment, model = optimize(
    parameters=[
        {
            "name": "weight_decay",
            "type": "choice",
            "values": [0.5, 1e-3],
            "value_type": "float"  # Optional, defaults to inference from type of "bounds".
            #"log_scale": False,  # Optional, defaults to False.
        }
#         {
#             "name": "batch_size",
#             "type": "choice",
#             "values": [256, 512],
#             "value_type": "int"
#         }
#         {
#             "name": "num_epochs",
#             "type": "choice",
#             "values": [1000, 2000],
#             "value_type": "int"
#         }
#         {
#             "name": "learning_rate",
#             "type": "range",
#             "bounds": [1e-5, 1e-3], #mortality1y
#             #"bounds": [1e-5, 1e-4], #diag
#             "value_type": "float",  # Optional, defaults to inference from type of "bounds".
#             "log_scale": False,  # Optional, defaults to False.
#         },
#         {
#             "name": "convg_thres",
#             "type": "range",
#             "bounds": [1e-5, 1e-3], #diag
#             "value_type": "float",  # Optional, defaults to inference from type of "bounds".
#             "log_scale": False,  # Optional, defaults to False.
#         }
        # {
        #     "name": "num_layers",
        #     "type": "choice",
        #     #"values": [0, 1, 2],
        #     "values": [2,2],
        #     "value_type": "int"
        # },
        # {
        #     "name": "k",
        #     "type": "choice",
        #     #"values": [50, 100, 150, 200],
        #     #"values": [50,100,200],
        #     "value_type": "int"
        # }
        # {
        #     "name": "actf",
        #     "type": "choice",
        #     "values": ["tanh", "sigma"],
        #     "value_type": "str"
        # }
        # {
        #     "name": "num_layers",
        #     "type": "choice",
        #     "values": [1,2],
        #     "value_type": "int"
        # }
    ],
    experiment_name="ncmf",
    objective_name="auc",
    evaluation_function=run_ncmf,
    minimize=False,  # Optional, defaults to False.
    #parameter_constraints=["k%2 <= 0"],  # Optional.
    #outcome_constraints=["loss >= 0"],  # Optional.
    total_trials=2, # Optional.
)


[INFO 07-21 15:52:15] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 07-21 15:52:15] ax.service.managed_loop: Started full optimization with 2 steps.
[INFO 07-21 15:52:15] ax.service.managed_loop: Running optimization trial 1...


Mapping node ids to matrix indices...
Splitting training and validation links...
Loading matrices and masks...
To reconstruct X0
dim:0; e0
X0 e0 row
X2 e0 col
dim:1; e1
X0 e1 col
X1 e1 row
To reconstruct X1
dim:0; e1
X0 e1 col
X1 e1 row
dim:1; e2
X1 e2 col
X2 e2 row
To reconstruct X2
dim:0; e2
X1 e2 col
X2 e2 row
dim:1; e0
X0 e0 row
X2 e0 col
Preparing autoencoders' configurations...
Preparing reconstructors' configurations...
Preparing fusions' configurations...
Initialising autoencoders...
Initialising reconstructors...
Initialising fusions...
Retreive Embedding
====> Epoch 0: Average Train Loss: 21.5427955 | Train RMSE: 5.1967854 | Average Valid Loss: 6.1572056 | Valid RMSE: 10.8265608 | beta: 0.0
Retreive Embedding
====> Epoch 1: Average Train Loss: 21.5145757 | Train RMSE: 5.1414132 | Average Valid Loss: 6.1444920 | Valid RMSE: 10.7673937 | beta: 0.0
Retreive Embedding
====> Epoch 2: Average Train Loss: 21.4887160 | Train RMSE: 5.0950060 | Average Valid Loss: 6.1317891 | Valid RMS

[INFO 07-21 15:53:19] ax.service.managed_loop: Running optimization trial 2...


Record Results!
DCMF++ eval done
#
ncmf_model.out_dict_info: 
{'F1': 0.7824033229481305,
 'auc': 0.808919435719484,
 'mrr': 0.8523750340489284,
 'params': {'autoencoder_config': {'activation_function': 'tanh',
                                   'hidden_dim': 1024,
                                   'k': 50,
                                   'k_factor': 0},
            'fusion_config': {'activation_function': 'tanh'},
            'hyperparameter_config': {'anneal': 'cosine',
                                      'convergence_threshold': 0.001,
                                      'lamda': 0.001,
                                      'learning_rate': 1e-06,
                                      'max_norm': 1,
                                      'ntrain_neg': 5,
                                      'num_cycles': 10,
                                      'num_epochs': 10,
                                      'nvalid_neg': 5,
                                      'pretrain': False,
  

In [11]:
#Info about all the ax trails
print("experiment.trials: ")
pprint.pprint(experiment.trials)
print("#")

experiment.trials: 
{0: Trial(experiment_name='ncmf', index=0, status=TrialStatus.COMPLETED, arm=Arm(name='0_0', parameters={'weight_decay': 0.001})),
 1: Trial(experiment_name='ncmf', index=1, status=TrialStatus.COMPLETED, arm=Arm(name='1_0', parameters={'weight_decay': 0.5}))}
#


In [12]:
# The best hyper-parameters found using ax
print("best_parameters: ")
print(best_parameters)
print("#")

best_parameters: 
{'weight_decay': 0.001}
#


In [13]:
#The loss corresponding to the best hyper-parameters
print("values[0]: ")
print(values[0])
print("#")

values[0]: 
{'auc': 0.808919435719484}
#


In [14]:
#The loss corresponding to all the hyper-parameters tried
for idx in experiment.trials.keys():
    trial =  experiment.trials[idx]
    print("obj: ",round(trial.objective_mean,4)," params: ",trial.arm.parameters)
print("#")

obj:  0.8089  params:  {'weight_decay': 0.001}
obj:  0.8089  params:  {'weight_decay': 0.5}
#


## Rerunning NCMF with the best parameters found using the ax framework

### *Instantiating the NCMF model with the best hyper-parameters*

In [15]:
# load the best hyper-parameters
weight_decay = best_parameters["weight_decay"]
learning_rate = 1e-6
convergence_threshold = 1e-3
num_epochs = 10
batch_size = 2048
entity_matrices = ['X0', 'X1', 'X2']
matrix_types = {
    "real": ["X0", "X1", "X2"],
    "binary": []
    }

In [16]:
ncmf_model = ncmf(sample_no, data_dir, dataset_name, matrix_types, num_epochs, learning_rate, weight_decay, convergence_threshold, batch_size, batch_size, entity_matrices)

#### *Fitting... *
- Performs the input transformation and network construction
- (Pre-trains and) trains the model to obtain the entity representations
- Reconstruct the input matrices using the entity representations obtained

In [17]:
ncmf_model.fit()

Mapping node ids to matrix indices...
Splitting training and validation links...
Loading matrices and masks...
To reconstruct X0
dim:0; e0
X0 e0 row
X2 e0 col
dim:1; e1
X0 e1 col
X1 e1 row
To reconstruct X1
dim:0; e1
X0 e1 col
X1 e1 row
dim:1; e2
X1 e2 col
X2 e2 row
To reconstruct X2
dim:0; e2
X1 e2 col
X2 e2 row
dim:1; e0
X0 e0 row
X2 e0 col
Preparing autoencoders' configurations...
Preparing reconstructors' configurations...
Preparing fusions' configurations...
Initialising autoencoders...
Initialising reconstructors...
Initialising fusions...
Retreive Embedding
====> Epoch 0: Average Train Loss: 21.5427955 | Train RMSE: 5.1967864 | Average Valid Loss: 6.1572085 | Valid RMSE: 10.8265661 | beta: 0.0
Retreive Embedding
====> Epoch 1: Average Train Loss: 21.5145743 | Train RMSE: 5.1414151 | Average Valid Loss: 6.1444913 | Valid RMSE: 10.7673998 | beta: 0.0
Retreive Embedding
====> Epoch 2: Average Train Loss: 21.4887170 | Train RMSE: 5.0950103 | Average Valid Loss: 6.1317898 | Valid RMS

In [18]:
ncmf_model.evaluate()

Start eval
Starting evaluation func
check1
check 2
check 3
Just before cross val
Start Evaluation Fold 0!
Start Evaluation Fold 1!
Start Evaluation Fold 2!
Start Evaluation Fold 3!
Start Evaluation Fold 4!
Record Results!
DCMF++ eval done


#### *Result attributes:*
- **out_dict_info**: dict, keys are loss/validation performance attributes and values are corresponding results.


In [19]:
ncmf_model.out_dict_info

{'params': {'hyperparameter_config': {'num_epochs': 10,
   'learning_rate': 1e-06,
   'weight_decay': 0.001,
   'convergence_threshold': 0.001,
   'train_batch_size': 2048,
   'valid_batch_size': 2048,
   'pretrain': False,
   'max_norm': 1,
   'lamda': 0.001,
   'anneal': 'cosine',
   'num_cycles': 10,
   'proportion': 0.8,
   'ntrain_neg': 5,
   'nvalid_neg': 5},
  'autoencoder_config': {'k': 50,
   'k_factor': 0,
   'hidden_dim': 1024,
   'activation_function': 'tanh'},
  'reconstructor_config': {'activation_function': 'tanh'},
  'fusion_config': {'activation_function': 'tanh'}},
 'auc': 0.8089306144330044,
 'mrr': 0.8523750340489284,
 'recall': 0.7655927201753346,
 'precision': 0.8001961415667695,
 'F1': 0.7824153092173912}